In [1]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

from Data_Collection import definitions
from Data_Collection import instantiation
from Data_Collection import detection
from Data_Collection import draw_landmarks
from Data_Collection import extract_keypoints

train = False

### Storing all the imported data

In [2]:
width_cam, height_cam, fps, actions, num_sequence, sequence_len, path = definitions()
mp_holistic, mp_drawing = instantiation()

### Prepocess Data and Create Labels and Features

In [3]:
if train:
    label_map = {label:num for num, label in enumerate(actions)}

    # Storing the videos (sequences) and labeling it with the respective action labels in one single large array
    sequences, labels = [], []
    for action in actions:
        for sequence in range(num_sequence):
            # The array 'window' stores all the frames for a particular sequence (video). 
            # This will have size of 30
            window = []
            for frame_num in range(sequence_len):
                res = np.load(os.path.join(path, action, str(sequence), "{}.npy".format(frame_num)))
                window.append(res)
            # The array 'sequences' stores all the videos for a particular action. 
            # This will have size of 90
            sequences.append(window)
            labels.append(label_map[action])

In [4]:
# The array 'sequences' stores all the videos and all the frames in that video for a particular action. 
# This will have shape of (90,30,1662) which means 90 videos, 30 frames per video, 1662 keypoints.

# [
#     [
#         [0th frame kepoints for 0th video of action 1], [1st frame keypoints for 0th video of action 1] ... [29th frame keypoint for 0th video of action 1]
#     ],
#     .
#     .
#     .
#     [
#         [0th frame kepoints for 29th video of action 1], [1st frame keypoints for 29th video of action 1] ... [29th frame keypoint for 29th video of action 1]
#     ]
    
#     This repeats 90 times total (30 videos of 3 actions)
# ]
# (np.array(sequences)).shape
# np.array(sequences)

In [5]:
if train:
    x = np.array(sequences)
    y = to_categorical(labels).astype(int)
    print("Shapes x and y")
    print(x.shape, y.shape)

    # Split the data into train-test data
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.10, shuffle=True)

    print("Shapes of x_train and x_test")
    print(x_train.shape, x_test.shape)
    print("Shapes of y_test")
    print(y_test)

Shapes x and y
(120, 30, 258) (120, 3)
Shapes of x_train and x_test
(108, 30, 258) (12, 30, 258)
Shapes of y_test
[[0 1 0]
 [0 0 1]
 [0 1 0]
 [1 0 0]
 [1 0 0]
 [1 0 0]
 [0 1 0]
 [0 1 0]
 [0 0 1]
 [1 0 0]
 [0 1 0]
 [1 0 0]]


### Build and Train LSTM Neural Network

In [6]:
from tensorflow.keras.models import Sequential # Build a sequential NN
from tensorflow.keras.layers import LSTM, Dense # LSTM and the dense layers 
from tensorflow.keras.callbacks import TensorBoard # To monitor NN training
import tensorflow
# LSTM requires lesser data, is faster to train and is able to detect faster

In [7]:
if train:
    log_dir = os.path.join('Training_Logs')
    tb_callback = TensorBoard(log_dir=log_dir)

    model = Sequential()
    model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(x.shape[1],x.shape[2])))
    model.add(LSTM(128, return_sequences=True, activation='relu'))
    model.add(LSTM(64, return_sequences=False, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(actions.shape[0], activation='softmax'))

    opt = tensorflow.keras.optimizers.Adam(learning_rate=0.001, beta_1=0.8,
                                           beta_2=0.899,
                                           epsilon=1e-07)
    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
    model.fit(x_train, y_train, epochs=50, callbacks=[tb_callback])

    print()
    model.summary()

Epoch 1/50
4/4 [==============================] - 3s 42ms/step - loss: 1.1468 - categorical_accuracy: 0.4167
Epoch 2/50
4/4 [==============================] - 0s 35ms/step - loss: 0.9966 - categorical_accuracy: 0.4815
Epoch 3/50
4/4 [==============================] - 0s 35ms/step - loss: 0.7557 - categorical_accuracy: 0.6296
Epoch 4/50
4/4 [==============================] - 0s 35ms/step - loss: 0.9650 - categorical_accuracy: 0.5463
Epoch 5/50
4/4 [==============================] - 0s 36ms/step - loss: 0.7518 - categorical_accuracy: 0.6019
Epoch 6/50
4/4 [==============================] - 0s 36ms/step - loss: 0.6787 - categorical_accuracy: 0.6296
Epoch 7/50
4/4 [==============================] - 0s 35ms/step - loss: 0.8279 - categorical_accuracy: 0.6111
Epoch 8/50
4/4 [==============================] - 0s 36ms/step - loss: 0.6799 - categorical_accuracy: 0.6019
Epoch 9/50
4/4 [==============================] - 0s 36ms/step - loss: 0.7078 - categorical_accuracy: 0.5556
Epoch 10/50
4/4 [==

### Save Weights

In [8]:
if train:
    model.save('action.h5')

### Testing a test-case

In [9]:
if train:
    res = model.predict(x_test)
    prediction_label = actions[np.argmax(res[3])]
    true_label = actions[np.argmax(y_test[3])]

    print(" Predicted Label: {}, \n True Label: {}".format(prediction_label, true_label))
    
    print(actions[np.argmax(res[0])], actions[np.argmax(y_test[0])])
    print (" y_test data ")
    print(y_test)

1/1 [==============================] - 0s 457ms/step
 Predicted Label: Hello, 
 True Label: Hello
Thanks Thanks
 y_test data 
[[0 1 0]
 [0 0 1]
 [0 1 0]
 [1 0 0]
 [1 0 0]
 [1 0 0]
 [0 1 0]
 [0 1 0]
 [0 0 1]
 [1 0 0]
 [0 1 0]
 [1 0 0]]


### Evaluation using Confusion Matrix and Accuracy

In [10]:
if train:
    from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

    y_predict = model.predict(x_test)

    # Converting the true labels from one-hot-encoded version to a list version with actual label numbers
    y_true = np.argmax(y_test, axis=1).tolist()

    # Converting the prdicted labels from one-hot-encoded version to a list version with actual label numbers
    y_predict = np.argmax(y_predict, axis=1).tolist()

    print("Confusion Matrix")
    print(multilabel_confusion_matrix(y_true, y_predict))
    print("The Accuracy Score is: ",accuracy_score(y_true, y_predict))

1/1 [==============================] - 0s 31ms/step
Confusion Matrix
[[[ 7  0]
  [ 0  5]]

 [[ 7  0]
  [ 0  5]]

 [[10  0]
  [ 0  2]]]
The Accuracy Score is:  1.0
